In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.losses import MeanSquaredError

# Load data
data = pd.read_csv("DATA.csv")

# Split features and target
X = data.iloc[:, :]  # Use all columns as features
y = data.iloc[:, :]  # Predict all columns

# Split data with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42
)

# Use RobustScaler to handle outliers better
scaler_X = RobustScaler()
scaler_y = RobustScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)


def build_model(input_dim, output_dim):
    model = Sequential()

    # Input layer
    model.add(Dense(256, activation="elu", kernel_regularizer=l1_l2(l1=1e-6, l2=1e-5)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    # First hidden block - larger capacity
    model.add(Dense(512, activation="elu", kernel_regularizer=l1_l2(l1=1e-6, l2=1e-5)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    # Second hidden block
    model.add(Dense(256, activation="elu", kernel_regularizer=l1_l2(l1=1e-6, l2=1e-5)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    # Third hidden block
    model.add(Dense(128, activation="elu", kernel_regularizer=l1_l2(l1=1e-6, l2=1e-5)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    # Fourth hidden block
    model.add(Dense(64, activation="elu", kernel_regularizer=l1_l2(l1=1e-6, l2=1e-5)))
    model.add(BatchNormalization())
    model.add(Dropout(0.15))

    # Output layer
    model.add(Dense(output_dim, activation="linear"))

    return model


# Initialize model with correct input shape
model = build_model(X_train_scaled.shape[1], y_train_scaled.shape[1])

# Custom learning rate schedule
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9
learning_rate_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps, decay_rate
)


# Define custom loss using tf.keras backend operations
@tf.function
def custom_loss(y_true, y_pred):
    # Calculate MSE manually
    mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=-1)
    # Weights for different parameters
    weights = tf.constant(
        [1.0, 1.0, 1.0, 1.0, 1.0, 1.5, 1.0, 2.0, 1.5, 1.0], dtype=tf.float32
    )
    # Apply weights to the MSE
    weighted_mse = mse * weights
    return tf.reduce_mean(weighted_mse)


optimizer = Adam(learning_rate=learning_rate_schedule)
model.compile(optimizer=optimizer, loss=custom_loss, metrics=["mae"])

# Enhanced callbacks
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=40,
    restore_best_weights=True,
    verbose=1,
    min_delta=1e-4,
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=15, min_lr=1e-6, verbose=1
)

# Train with optimal parameters
history = model.fit(
    X_train_scaled,
    y_train_scaled,
    epochs=500,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1,
)

# Evaluate
test_loss = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
print(f"\nTest Loss: {test_loss:.4f}")

# Save model
model.save("water_quality_ann_enhanced.h5")

# Visualize results
import matplotlib.pyplot as plt


def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Plot loss
    ax1.plot(history.history["loss"], label="Training Loss")
    ax1.plot(history.history["val_loss"], label="Validation Loss")
    ax1.set_title("Model Loss")
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss")
    ax1.legend()
    ax1.grid(True)

    # Plot MAE
    ax2.plot(history.history["mae"], label="Training MAE")
    ax2.plot(history.history["val_mae"], label="Validation MAE")
    ax2.set_title("Model MAE")
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("MAE")
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.savefig("training_metrics.png")
    plt.close()


plot_training_history(history)

# Make predictions and analyze results
predictions = model.predict(X_test_scaled)
predictions = scaler_y.inverse_transform(predictions)
actual = scaler_y.inverse_transform(y_test_scaled)

# Calculate and print R-squared for each parameter
for i, column in enumerate(data.columns):
    correlation_matrix = np.corrcoef(actual[:, i], predictions[:, i])
    r_squared = correlation_matrix[0, 1] ** 2
    print(f"{column} R-squared: {r_squared:.4f}")

# Print mean absolute percentage error for each parameter
mape = np.mean(np.abs((actual - predictions) / actual), axis=0) * 100
for i, column in enumerate(data.columns):
    print(f"{column} MAPE: {mape[i]:.2f}%")

Epoch 1/500


2025-01-30 21:10:32.599736: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [10] vs. [16]
	 [[{{function_node __forward_custom_loss_56735}}{{node mul}}]]


InvalidArgumentError: Graph execution error:

Detected at node mul defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/sic/Documentations/Shanky/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/sic/Documentations/Shanky/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/sic/Documentations/Shanky/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/sic/Documentations/Shanky/.venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/sic/Documentations/